# SITCOM-1089 - Strong Vibration Analysis

[SITCOM-1089]: https://jira.lsstcorp.org/browse/SITCOM-1089

In [ ]:
import sys, time, os, asyncio

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

from astropy.time import Time
from matplotlib.ticker import FormatStrFormatter
from scipy import signal
from scipy.signal import find_peaks
from lsst.ts.xml.tables.m1m3 import HP_COUNT

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import (
    getEfdData,
    makeEfdClient,
    clipDataToEvent,
    calcNextDay,
)

%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import sitcom1089

In [ ]:
begin_time = Time("2023-06-28 01:08:00", format="iso", scale="utc")
end_time = Time("2023-06-28 01:20:00", format="iso", scale="utc")
raised_begin = Time("2023-06-28 01:08:15", format="iso", scale="utc")
raised_end = Time("2023-06-28 01:11:55", format="iso", scale="utc")
lowering_end = Time("2023-06-28 01:15:15", format="iso", scale="utc")
down_end = Time("2023-06-28 01:20:00", format="iso", scale="utc")

time_dict = {}
time_dict["total"] = {"begin": begin_time, "end": end_time}
time_dict["raised"] = {"begin": begin_time, "end": raised_end}
time_dict["lowering"] = {"begin": raised_end, "end": lowering_end}
time_dict["down"] = {"begin": lowering_end, "end": end_time}


MEASURED_FORCES_TOPICS = [f"measuredForce{i}" for i in range(HP_COUNT)]

In [ ]:
from lsst_efd_client import EfdClient

client = EfdClient("idf_efd")

efd_dict = sitcom1089.get_efd_data(begin_time, end_time, client)

# Plots organized as follows
Telemetry
1. TMA telemetry
2. TMA telemetry Zoomed in
3. IMS position telemetry
4. IMS rotation telemetry
5. hardpoint measured force telemetry
6. hardpoint measured force telemetry zoomed in

PSDs
1. TMA torque psd
2. IMS position psd
3. TODO IMS rotaton
4. Hardpoint psd


In [ ]:
## Telemetry 

In [ ]:
# TMA telemetry
fig, axs = plt.subplots(2, dpi=125, figsize=(12, 3), sharex=True)
for i, key in enumerate(["actualTorque", "actualPosition"]):
    scale = 1
    if key == "actualTorque":
        scale = 1e3
        axs[i].set_ylabel(key + "\n [kN]")
    elif key == "actualPosition":
        scale = 1
        axs[i].set_ylabel(key + "\n [deg]")
    else:
        axs[i].set_ylabel(key)
    vals = efd_dict["el"][key] / scale
    x, y = sitcom1089.resample_times(efd_dict["el"]["timestamp"], vals, 0.02)
    times = Time(x, format="unix_tai", scale="utc").datetime
    axs[i].scatter(
        times,
        y,
        label=key,
        lw=0.3,
    )
    axs[i].legend(ncol=3, loc=9)
axs[i].set_xlabel("Time")

axs[0].set_title(
    f"TMA Elevation\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}"
)
axs[0].set_ylim(-500, 600)
_ = axs[0].set_yticks(np.arange(-500, 750, 250))

In [ ]:
# TMA telemetry Zoom
fig, axs = plt.subplots(2, dpi=125, figsize=(12, 3), sharex=True)
for i, key in enumerate(["actualTorque", "actualPosition"]):
    scale = 1
    if key == "actualTorque":
        scale = 1e3
        axs[i].set_ylabel(key + "\n [kN]")
    elif key == "actualPosition":
        scale = 1
        axs[i].set_ylabel(key + "\n [deg]")
    else:
        axs[i].set_ylabel(key)
    vals = efd_dict["el"][key] / scale
    x, y = sitcom1089.resample_times(efd_dict["el"]["timestamp"], vals, 0.02)
    times = Time(x, format="unix_tai", scale="utc").datetime
    axs[i].scatter(
        times,
        y,
        label=key,
        lw=0.3,
    )
    axs[i].legend(ncol=3, loc=9)
axs[i].set_xlabel("Time")

axs[0].set_title(
    f"TMA Elevation Zoom\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}"
)
axs[0].set_ylim(-500, 600)
axs[0].set_yticks(np.arange(-500, 750, 250))
mintime = 3000
maxtime = 3150
sel = (times > times[mintime]) & (times < times[maxtime])
maxpos = y[sel].max()
minpos = y[sel].min()
axs[1].set_ylim(minpos - 0.025, maxpos + 0.025)
axs[1].set_xlim(times[mintime], times[maxtime])
axs[1].set_yticks(np.arange(np.round(minpos - 0.025, 3), maxpos + 0.025, 0.025))

In [ ]:
# IMS position telemetry
times = Time(efd_dict["ims"]["timestamp"], format="unix_tai", scale="utc").datetime
for key in ["xPosition", "yPosition", "zPosition"]:
    plt.plot(
        times, (efd_dict["ims"][key] - efd_dict["ims"][key].mean()) * 1e3, label=key
    )

plt.axvline(
    time_dict["raised"]["end"].datetime, label="begin lowering", c="k", ls="dashdot"
)
plt.axvline(
    time_dict["lowering"]["end"].datetime, label="end lowering", c="k", ls="dotted"
)
plt.legend()
plt.xlabel("Time")
plt.ylabel("$\Delta$ postion [mm]")
plt.title(
    f"IMS Position\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}"
)

In [ ]:
# IMS rotation telemetry
times = Time(efd_dict["ims"]["timestamp"], format="unix_tai", scale="utc").datetime
for key in ["xRotation", "yRotation", "zRotation"]:
    plt.plot(
        times, np.rad2deg(efd_dict["ims"][key] - efd_dict["ims"][key].mean()), label=key
    )

plt.axvline(
    time_dict["raised"]["end"].datetime, label="begin lowering", c="k", ls="dashdot"
)
plt.axvline(
    time_dict["lowering"]["end"].datetime, label="end lowering", c="k", ls="dotted"
)
plt.legend()
plt.xlabel("Time")
plt.ylabel("$\Delta$ rotation [deg]")
plt.title(
    f"IMS Rotation\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}"
)

In [ ]:
# hardpoint measured force telemetry

plt.figure(dpi=125, figsize=(12, 4))
times = Time(efd_dict["hp"]["timestamp"], format="unix_tai", scale="utc").datetime
for key in MEASURED_FORCES_TOPICS:
    plt.plot(times, (efd_dict["hp"][key] - efd_dict["hp"][key].mean()), label=key)

plt.axvline(
    time_dict["raised"]["end"].datetime, label="begin lowering", c="k", ls="dashdot"
)
plt.axvline(
    time_dict["lowering"]["end"].datetime, label="end lowering", c="k", ls="dotted"
)
plt.legend(ncol=4)
plt.xlabel("Time")
plt.ylabel("$\Delta$ force [N]")
plt.title(
    f"hardpoint measured forces\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}"
)
plt.ylim(-3e3, 3e3)

In [ ]:
# hardpoint measured force telemetry zoom

plt.figure(dpi=125, figsize=(12, 4))
times = Time(efd_dict["hp"]["timestamp"], format="unix_tai", scale="utc").datetime
for key in MEASURED_FORCES_TOPICS:
    plt.plot(times, (efd_dict["hp"][key] - efd_dict["hp"][key].mean()), label=key)

plt.axvline(
    time_dict["raised"]["end"].datetime, label="begin lowering", c="k", ls="dashdot"
)
plt.axvline(
    time_dict["lowering"]["end"].datetime, label="end lowering", c="k", ls="dotted"
)
plt.legend(ncol=4)
plt.xlabel("Time")
plt.ylabel("$\Delta$ force [N]")
plt.title(
    f"Hardpoint Measured forces zoom\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}"
)
plt.ylim(-1e3, 1e3)
_ = plt.xlim(times[int(120 / 0.02)], times[int(130 / 0.02)])

In [ ]:
# TMA torque psd
dict_key = "el"
key = "actualTorque"
xvals = efd_dict[dict_key]["timestamp"]
yvals = efd_dict[dict_key][key]
delta_t = 0.02
xvals, yvals = sitcom1089.resample_times(xvals, yvals, delta_t)
times = Time(xvals, format="unix_tai", scale="utc")
fs = 1 / delta_t

xpeaks_total = [
    1.3,
    2.5,
    3.8,
    5.1,
    6.3,
    8.9,
    11.1,
    13.7,
    14.9,
    16.2,
    17.5,
    18.7,
    21.3,
    22.5,
    23.8,
]

plt.figure(dpi=125, figsize=(12, 4))
for step, time_key in enumerate(time_dict.keys()):
    sel = times > time_dict[time_key]["begin"]
    sel &= times <= time_dict[time_key]["end"]
    sel &= ~np.isnan(yvals)
    vals = yvals[sel]

    freq, psd = sitcom1089.get_freq_psd(vals, 1 / fs)
    psd_vals = (
        pd.DataFrame({"psd": psd})["psd"].rolling(int(10 / delta_t * 0.01)).mean()
    )
    normval = np.exp(
        np.polyval(
            np.polyfit(
                freq[~np.isnan(psd_vals)], np.log(psd_vals[~np.isnan(psd_vals)]), deg=4
            ),
            freq,
        )
    )
    xpeaks = sitcom1089.get_peak_points(
        freq,
        np.log(psd_vals / normval),
        height=2,
    )
    print(f"{time_key} identified peaks:")
    print([float(f"{i:0.1f}") for i in xpeaks])

    plt.plot(freq, psd_vals * 10 ** (-3 * step), zorder=9, lw=1, label=time_key)
    for peak in xpeaks_total:
        plt.axvline(peak, c="k", ls="dashed", zorder=1, lw=0.5)
    # plt.scatter(xpeaks, np.ones_like(xpeaks) * 1e19, c="red")
    # plt.plot(freq, normval)
plt.yscale("log")
# plt.ylim(1e9, 1e20)
plt.legend()
plt.xlabel("Frequency (Hz)")
plt.ylabel("power")
plt.title(
    f"TMA Elevation {key}\nEvent {begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}"
)
plt.xlim(0, 25)
_ = plt.xticks(np.arange(0, 26, 2))
# for i in [1.25, 2.5, 3.8]:
#     plt.axvline(i)
# plt.scatter(
#     freq,
#     ,
# )

In [ ]:
dict_key = "ims"
fig, axs = plt.subplots(3, 1, dpi=125, figsize=(14, 10), sharex=True)
plt.suptitle(
    f"Event {begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}", y=0.9
)
for i, key in enumerate(["xPosition", "yPosition", "zPosition"]):
    ax = axs[i]

    xvals = efd_dict[dict_key]["timestamp"]
    yvals = efd_dict[dict_key][key]
    delta_t = 0.02
    xvals, yvals = sitcom1089.resample_times(xvals, yvals, delta_t)
    times = Time(xvals, format="unix_tai", scale="utc")
    fs = 1 / delta_t
    for peak in xpeaks_total:
        axs[i].axvline(peak, c="k", ls="dashed", zorder=1, lw=0.5)
    for step, time_key in enumerate(time_dict.keys()):
        sel = times > time_dict[time_key]["begin"]
        sel &= times <= time_dict[time_key]["end"]
        sel &= ~np.isnan(yvals)
        vals = yvals[sel]

        freq, psd = sitcom1089.get_freq_psd(vals, 1 / fs)
        psd_vals = (
            pd.DataFrame({"psd": psd})["psd"].rolling(int(10 / delta_t * 0.01)).mean()
        )
        ax.plot(freq, psd_vals * 10 ** (-5 * step), zorder=9, lw=1, label=time_key)
    title_str = f"IMS {key}"
    ax.set(yscale="log", ylabel="power")
    ax.set_title(title_str, y=0.85)
    # plt.ylim(1e11,1e17)
axs[0].legend(ncol=4)
axs[2].set_xlabel("Frequency (Hz)")
axs[2].set_xticks(np.arange(0, 26, 2))
plt.subplots_adjust(hspace=0)
plt.savefig("./1089_data/ims_psd.png")

In [ ]:
# TODO ims rotation


# dict_key = "ims"
# fig, axs = plt.subplots(3, 1, dpi=125, figsize=(14, 10), sharex=True)
# plt.suptitle(
#     f"Event {begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}", y=0.9
# )
# for i, key in enumerate(["xPosition", "yPosition", "zPosition"]):
#     ax = axs[i]

#     xvals = efd_dict[dict_key]["timestamp"]
#     yvals = efd_dict[dict_key][key]
#     delta_t = 0.02
#     xvals, yvals = sitcom1089.resample_times(xvals, yvals, delta_t)
#     times = Time(xvals, format="unix_tai", scale="utc")
#     fs = 1 / delta_t
#     for peak in xpeaks_total:
#         axs[i].axvline(peak, c="k", ls="dashed", zorder=1, lw=0.5)
#     for step, time_key in enumerate(time_dict.keys()):
#         sel = times > time_dict[time_key]["begin"]
#         sel &= times <= time_dict[time_key]["end"]
#         sel &= ~np.isnan(yvals)
#         vals = yvals[sel]

#         freq, psd = sitcom1089.get_freq_psd(vals, 1 / fs)
#         psd_vals = (
#             pd.DataFrame({"psd": psd})["psd"].rolling(int(10 / delta_t * 0.01)).mean()
#         )
#         ax.plot(freq, psd_vals * 10 ** (-5 * step), zorder=9, lw=1, label=time_key)
#     title_str = f"IMS {key}"
#     ax.set(yscale="log", ylabel="power")
#     ax.set_title(title_str, y=0.85)
#     # plt.ylim(1e11,1e17)
# axs[0].legend(ncol=4)
# axs[2].set_xlabel("Frequency (Hz)")
# axs[2].set_xticks(np.arange(0, 26, 2))
# plt.subplots_adjust(hspace=0)
# plt.savefig("./1089_data/ims_psd.png")

In [ ]:
# hardpoint forces


dict_key = "hp"
fig, axs = plt.subplots(3, 1, dpi=125, figsize=(12, 7), sharex=True)
plt.suptitle(
    f"Hardpoints Event {begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}",
    y=0.9,
)
for i, key in enumerate(MEASURED_FORCES_TOPICS[:3]):
    ax = axs[i]

    xvals = efd_dict[dict_key]["timestamp"]
    yvals = efd_dict[dict_key][key]
    delta_t = 0.02
    xvals, yvals = sitcom1089.resample_times(xvals, yvals, delta_t)
    times = Time(xvals, format="unix_tai", scale="utc")
    fs = 1 / delta_t
    for peak in xpeaks_total:
        axs[i].axvline(peak, c="k", ls="dashed", zorder=1, lw=0.5)
    for step, time_key in enumerate(time_dict.keys()):
        sel = times > time_dict[time_key]["begin"]
        sel &= times <= time_dict[time_key]["end"]
        sel &= ~np.isnan(yvals)
        vals = yvals[sel]

        freq, psd = sitcom1089.get_freq_psd(vals, 1 / fs)
        psd_vals = (
            pd.DataFrame({"psd": psd})["psd"].rolling(int(10 / delta_t * 0.01)).mean()
        )
        ax.plot(freq, psd_vals * 10 ** (-5 * step), zorder=9, lw=1, label=time_key)
    title_str = f"{key}"
    ax.set(yscale="log", ylabel="power")
    ax.set_title(title_str, y=0.85)
    # plt.ylim(1e11,1e17)
axs[0].legend(ncol=4)
axs[2].set_xlabel("Frequency (Hz)")
axs[2].set_xticks(np.arange(0, 26, 2))
plt.subplots_adjust(hspace=0)